# Treinamento de uma rede neural Feed Foward para previsão de consumo de cimento 

## Arquitetura 

Consideramos cada estado, em cada mês de 1991 até 2019, como uma entrada separada do modelo

### Carregando os dados

Utilizamos o módulo `load_data` com os métodos `get_df` e `get_consumo`

In [ ]:
def init ():

  global df, anomes

  # variaveis auxiliares
  anos = [str(i) for i in range(1991, 2020)]
  meses = ["{:02d}".format(i) for i in range(1,13)] # colocamos os 0 a esquerda se precisar
  anomes = []
  n_estados = 27

  for ano in anos:
    anomes += [str(ano) + i for i in meses]

  df = [[i] for i in range(1,28)] * len(anomes)

# funcao que pega um pandas dataframe e coloca os valor no array que sera usado como entrada para o  modelo
def df_to_list(indicador, array):

  global anomes
  
  index = 0 

  for coluna in anomes:

    for linha in indicador[coluna]:

      array[index] = array[index] + [linha]
      index += 1
  
  return array

# Carrega os dados de consumo de cimento 
# do PATH considerando a 
def get_consumo (path : str, var : float) -> list :

  import pandas as pd

  init()

  consumo = pd.read_csv(path + "consumo.csv")

  anomes = ["199012"] + anomes

  tendencia = []

  # anomes de 19901 ate 201912
  for index  in range(1,len(anomes)):

    # estados...
    for estado in consumo.index:

      tend = 1 - (consumo[anomes[index]][estado]/consumo[anomes[index - 1]][estado])

      if tend < - var:
        tendencia += [[1., 0., 0.]]

      elif tend <= var:
        tendencia += [[0., 1., 0.]]

      else:
        tendencia += [[0., 0., 1.]]

  anomes = anomes[1:]

  return tendencia

def get_df (path : str):

  import pandas as pd

  init()

  global df

  # PIB a precos constantes
  pib_cte = pd.read_csv(path + "pib_precos_constantes.csv")
  df = df_to_list(pib_cte, df)

  # PIB a precos de mercado corrente 
  pib_pm = pd.read_csv(path + "pib_precos_mercado_corrente.csv")
  df = df_to_list(pib_pm, df)

  # PIB a precos de mercado corrente 
  pib_pc = pd.read_csv(path + "pib_per_capita.csv")
  df = df_to_list(pib_pc, df)

  # PIB a precos de mercado corrente 
  pib_cc = pd.read_csv(path + "pib_construcao.csv")
  df = df_to_list(pib_cc, df)

  pop = pd.read_csv(path + "populacao.csv")
  df = df_to_list(pop, df)

  ipca = pd.read_csv(path + "IPCA.csv")
  df = df_to_list(ipca, df)

  incc = pd.read_csv(path + "INCC.csv")
  df = df_to_list(incc, df)

  selic = pd.read_csv(path + "SELIC.csv")
  df = df_to_list(selic, df)

  # IDH Renda
  idh_renda = pd.read_csv(path + "idh_renda.csv")
  df = df_to_list(idh_renda, df)

  # IDH Saúde
  idh_saude = pd.read_csv(path + "idh_saude.csv")
  df = df_to_list(idh_saude, df)

  # IDH Educação
  idh_educacao = pd.read_csv(path + "idh_educacao.csv")
  df = df_to_list(idh_educacao, df)

  return df

In [ ]:
# Parâmetros

PATH = "/content/drive/MyDrive/TCC/dados/anualizados/csv/"
VAR = 0.005

In [ ]:
# Dados do consumo 
# from load_data import get_consumo 
tendencia = get_consumo(PATH, VAR)

In [ ]:
# Dados de entrada 
# from load_data import get_df 
df = get_df(PATH)

## Treinamento do modelo

Vamos treinar uma rede neural utilizando a biblioteca TensorFlow

Dividimos os dados em:
- 70%: treino
- 30%: teste

In [ ]:
# Importando as libs

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [ ]:
# separando conjuntos de treino e teste (70% e 30%)
X_train, X_test, y_train, y_test = train_test_split(df, tendencia,test_size=0.2)

#### Definindo o modelo 

- `input_shape`: quantas variáveis em cada entrada
- `activation`: função de ativação, vamos usar a `soft_max`

In [ ]:
#vamos definir o modelo
model = keras.Sequential()
model.add(Dense(12, input_shape=(12,), activation="softmax"))
model.add(Dense(3, activation="softmax"))

#### Compilando o modelo

Otimizador:
- https://analyticsindiamag.com/guide-to-tensorflow-keras-optimizers/


In [ ]:
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])


#### Treinando o modelo

Usamos 10 epochs

In [ ]:
model.fit(X_train, y_train, epochs=8) 

Epoch 1/8
235/235 [==============================] - 1s 2ms/step - loss: 1.0856 - categorical_accuracy: 0.4089
Epoch 2/8
235/235 [==============================] - 0s 2ms/step - loss: 1.0503 - categorical_accuracy: 0.4899
Epoch 3/8
235/235 [==============================] - 0s 2ms/step - loss: 1.0438 - categorical_accuracy: 0.4899
Epoch 4/8
235/235 [==============================] - 0s 2ms/step - loss: 1.0416 - categorical_accuracy: 0.4899
Epoch 5/8
235/235 [==============================] - 0s 2ms/step - loss: 1.0406 - categorical_accuracy: 0.4899
Epoch 6/8
235/235 [==============================] - 0s 2ms/step - loss: 1.0400 - categorical_accuracy: 0.4899
Epoch 7/8
235/235 [==============================] - 0s 2ms/step - loss: 1.0398 - categorical_accuracy: 0.4899
Epoch 8/8
235/235 [==============================] - 0s 2ms/step - loss: 1.0395 - categorical_accuracy: 0.4899


In [ ]:
# Avaliacao
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

59/59 - 0s - loss: 1.0466 - categorical_accuracy: 0.4824 - 226ms/epoch - 4ms/step


In [ ]:
# previsao
predictions = model.predict(X_test)

In [ ]:
predictions[0]

array([0.2802592 , 0.488388  , 0.23135282], dtype=float32)

In [ ]:
# transforma dado de [0.85, 0.10,0.05]
# em [-1]
def processa_tendencia (dado):

  lis = []

  for data in dado:

    res = data.index(max(data))
    # Queda
    if res == 0:
      lis += [-1]
    # Estabilidade
    elif res == 1:
      lis += [0]
    # Aumento
    else:
      lis += [1]

  return lis

In [ ]:
pred = [list(i) for i in predictions]

pp = processa_tendencia(pred)
pr = processa_tendencia(y_test)

In [ ]:
acertos = 0
erros = 0

for i in range(len(pp)):
  if pp[i] == pr[i]:
    acertos += 1
  else:
    erros += 1

print(f"Acertos: {acertos} -> {acertos/len(pp)}%")
print(f"Erros: {erros} -> {erros/len(pp)}%")


Acertos: 907 -> 0.4824468085106383%
Erros: 973 -> 0.5175531914893617%


In [ ]:
acertos = 0
erros = 0
queda = 0

for i in range(len(pp)):
  if pr[i] == -1:
    queda +=1
  if pp[i] == pr[i] and pr[i] == -1:
    acertos += 1
  else:
    erros += 1

print(f"Acertos: {acertos} -> {acertos/queda}%")
print(f"Erros: {erros} -> {erros/len(pp)}%")


Acertos: 0 -> 0.0%
Erros: 1880 -> 1.0%


In [ ]:
acertos = 0
erros = 0
estab = 0

for i in range(len(pp)):
  if pr[i] == 0:
    estab +=1
  if pp[i] == pr[i] and pr[i] == 0:
    acertos += 1
  else:
    erros += 1

print(f"Acertos: {acertos} -> {acertos/estab}%")
print(f"Erros: {erros} -> {erros/len(pp)}%")


Acertos: 918 -> 1.0%
Erros: 962 -> 0.5117021276595745%


In [ ]:
acertos = 0
erros = 0
aumento = 0

for i in range(len(pp)):
  if pr[i] == 0:
    aumento +=1
  if pp[i] == pr[i] and pr[i] == 1:
    acertos += 1
  else:
    erros += 1

print(f"Acertos: {acertos} -> {acertos/aumento}%")
print(f"Erros: {erros} -> {erros/len(pp)}%")


Acertos: 0 -> 0.0%
Erros: 940 -> 1.0%


In [ ]:
# 1) 075%

# Dados do consumo 
from load_data import get_consumo 
tend_075 = get_consumo(PATH, 0.0075)

# separando conjuntos de treino e teste (70% e 30%)
X_train, X_test, y_train, y_test = train_test_split(df, tend_075,test_size=0.3)

#vamos definir o modelo
model_075 = keras.Sequential()
model_075.add(Dense(12, input_shape=(12,), activation="softmax"))
model_075.add(Dense(3, activation="softmax"))

model_075.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

model_075.fit(X_train, y_train, epochs=10) 

# Avaliacao
test_loss, test_acc = model_075.evaluate(X_test,  y_test, verbose=2)

predictions075 = model_075.predict(X_test)

pred075 = [list(i) for i in predictions075]
#pred = processa_tendencia(p)

Epoch 1/10
206/206 [==============================] - 1s 1ms/step - loss: 1.1123 - categorical_accuracy: 0.3568
Epoch 2/10
206/206 [==============================] - 0s 1ms/step - loss: 1.0893 - categorical_accuracy: 0.3549
Epoch 3/10
206/206 [==============================] - 0s 1ms/step - loss: 1.0871 - categorical_accuracy: 0.3798
Epoch 4/10
206/206 [==============================] - 0s 1ms/step - loss: 1.0870 - categorical_accuracy: 0.3798
Epoch 5/10
206/206 [==============================] - 0s 1ms/step - loss: 1.0870 - categorical_accuracy: 0.3798
Epoch 6/10
206/206 [==============================] - 0s 1ms/step - loss: 1.0870 - categorical_accuracy: 0.3798
Epoch 7/10
206/206 [==============================] - 0s 1ms/step - loss: 1.0870 - categorical_accuracy: 0.3798
Epoch 8/10
206/206 [==============================] - 0s 1ms/step - loss: 1.0870 - categorical_accuracy: 0.3798
Epoch 9/10
206/206 [==============================] - 0s 1ms/step - loss: 1.0869 - categorical_accuracy:

In [ ]:
pp_075 = processa_tendencia(pred075)

acertos = 0
erros = 0
estab = 0

for i in range(len(pp_075)):
  if pp_075[i] == pr[i]:
    acertos += 1
    if pp_075[i] == 0:
      estab += 1
  else:
    erros += 1

print(f"Acertos: {acertos} -> {acertos/len(pp_075)}%")
print(f"Erros: {erros} -> {erros/len(pp_075)}%")
print(f"Acertos quando chutou estabilidade: {estab} -> {estab/acertos}%")

Acertos: 1386 -> 0.49166371053565094%
Erros: 1433 -> 0.5083362894643491%
Acertos quando chutou estabilidade: 1386 -> 1.0%


In [ ]:
# 1) 075%

# Dados do consumo 
from load_data import get_consumo 
tend_05 = get_consumo(PATH, 0.005)

# separando conjuntos de treino e teste (70% e 30%)
X_train, X_test, y_train, y_test = train_test_split(df, tend_05,test_size=0.3)

#vamos definir o modelo
model_05 = keras.Sequential()
model_05.add(Dense(12, input_shape=(12,), activation="softmax"))
model_05.add(Dense(3, activation="softmax"))

model_05.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

model_05.fit(X_train, y_train, epochs=10) 

# Avaliacao
test_loss, test_acc = model_05.evaluate(X_test,  y_test, verbose=2)

predictions05 = model_05.predict(X_test)

pred05 = [list(i) for i in predictions05]
#pred = processa_tendencia(p)

Epoch 1/10
206/206 [==============================] - 1s 1ms/step - loss: 1.1079 - categorical_accuracy: 0.3474
Epoch 2/10
206/206 [==============================] - 0s 1ms/step - loss: 1.0845 - categorical_accuracy: 0.4026
Epoch 3/10
206/206 [==============================] - 0s 1ms/step - loss: 1.0800 - categorical_accuracy: 0.4128
Epoch 4/10
206/206 [==============================] - 0s 1ms/step - loss: 1.0787 - categorical_accuracy: 0.4168
Epoch 5/10
206/206 [==============================] - 0s 1ms/step - loss: 1.0781 - categorical_accuracy: 0.4168
Epoch 6/10
206/206 [==============================] - 0s 1ms/step - loss: 1.0776 - categorical_accuracy: 0.4168
Epoch 7/10
206/206 [==============================] - 0s 1ms/step - loss: 1.0773 - categorical_accuracy: 0.4168
Epoch 8/10
206/206 [==============================] - 0s 1ms/step - loss: 1.0772 - categorical_accuracy: 0.4168
Epoch 9/10
206/206 [==============================] - 0s 1ms/step - loss: 1.0771 - categorical_accuracy:

In [ ]:
pp_05 = processa_tendencia(pred05)

acertos = 0
erros = 0
estab = 0

for i in range(len(pp_05)):
  if pp_05[i] == pr[i]:
    acertos += 1
    if pp_05[i] == 0:
      estab += 1
  else:
    erros += 1

print(f"Acertos: {acertos} -> {acertos/len(pp_05)}%")
print(f"Erros: {erros} -> {erros/len(pp_05)}%")
print(f"Acertos quando chutou estabilidade: {estab} -> {estab/acertos}%")

Acertos: 795 -> 0.2820148989003193%
Erros: 2024 -> 0.7179851010996807%
Acertos quando chutou estabilidade: 0 -> 0.0%


#### Análises e otimização da margem de estabilidade

In [ ]:
# Mostra as estatisticas
def show_stat (ind, size):
  
  print("Nos dados, temos:")
  print(f"Queda: {ind[-1][1]} - {(ind[-1][1] / size)*100:.1f}%")
  print(f"Estabilidade: {ind[0][1]} - {(ind[0][1] / size)*100:.1f}%")
  print(f"Aumento: {ind[1][1]} - {100*(ind[1][1] / size):.1f}%")

In [ ]:
def get_ind (dado):

  count = {-1: ["Queda", 0], 
            0: ["Estabilidade",0], 
            1: ["Aumento",0]}

  for x in dado:
    count[x][1] += 1

  return count

In [ ]:
n_anomes = 348
n_estados = 27

In [ ]:
# y_test
print("Estatísticas y_test")
show_stat(get_ind(processa_tendencia(y_test)),len(y_test))

Estatísticas y_test
Nos dados, temos:
Queda: 260 - 9.2%
Estabilidade: 2342 - 83.1%
Aumento: 217 - 7.7%


In [ ]:
# pred
pred = [list(i) for i in predictions]
print("Estatísticas reais")
show_stat(get_ind(processa_tendencia(pred)),len(predictions))

Estatísticas reais
Nos dados, temos:
Queda: 0 - 0.0%
Estabilidade: 2819 - 100.0%
Aumento: 0 - 0.0%


In [ ]:
# dados completos
print("Estatísticas completas")
show_stat(get_ind(processa_tendencia(tendencia)),len(tendencia))

Estatísticas completas
Nos dados, temos:
Queda: 815 - 8.7%
Estabilidade: 7891 - 84.0%
Aumento: 690 - 7.3%


#### testando com outros valores de estabilidade


In [ ]:
# t = 0.05%
print("Previsao com t = 0.05%")
show_stat(get_ind(processa_tendencia(pred05)),len(pred05))

print("----------------------")

# dados completos
print("Estatísticas completas")
show_stat(get_ind(processa_tendencia(tend_05)),len(tend_05))


Previsao com t = 0.05%
Nos dados, temos:
Queda: 2548 - 90.4%
Estabilidade: 0 - 0.0%
Aumento: 271 - 9.6%
----------------------
Estatísticas completas
Nos dados, temos:
Queda: 4351 - 46.3%
Estabilidade: 2090 - 22.2%
Aumento: 2955 - 31.4%


In [ ]:
# 1) 1%

# Dados do consumo 
from load_data import get_consumo 
tend_1 = get_consumo(PATH, 0.01)

# dados completos
print("Estatísticas completas [t=1%]")
show_stat(get_ind(processa_tendencia(tend_1)),len(tend_1))

# separando conjuntos de treino e teste (70% e 30%)
X_train, X_test, y_train, y_test = train_test_split(df, tend_1,test_size=0.3)

#vamos definir o modelo
model = keras.Sequential()
model.add(Dense(12, input_shape=(12,), activation="softmax"))
model.add(Dense(3, activation="softmax"))

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

model.fit(X_train, y_train, epochs=10) 

# Avaliacao
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

predictions1 = model.predict(X_test)

pred1 = [list(i) for i in predictions1]
#pred = processa_tendencia(p)

Estatísticas completas [t=1%]
Nos dados, temos:
Queda: 3248 - 34.6%
Estabilidade: 3982 - 42.4%
Aumento: 2166 - 23.1%
Epoch 1/10
206/206 [==============================] - 1s 2ms/step - loss: 1.0800 - categorical_accuracy: 0.4209
Epoch 2/10
206/206 [==============================] - 0s 2ms/step - loss: 1.0712 - categorical_accuracy: 0.4209
Epoch 3/10
206/206 [==============================] - 0s 2ms/step - loss: 1.0704 - categorical_accuracy: 0.4209
Epoch 4/10
206/206 [==============================] - 0s 2ms/step - loss: 1.0704 - categorical_accuracy: 0.4209
Epoch 5/10
206/206 [==============================] - 0s 2ms/step - loss: 1.0703 - categorical_accuracy: 0.4209
Epoch 6/10
206/206 [==============================] - 0s 2ms/step - loss: 1.0704 - categorical_accuracy: 0.4209
Epoch 7/10
206/206 [==============================] - 0s 2ms/step - loss: 1.0703 - categorical_accuracy: 0.4209
Epoch 8/10
206/206 [==============================] - 0s 2ms/step - loss: 1.0703 - categorical_accu

In [ ]:
# t = 0.05%
print("Previsao com t = 0.1%")
show_stat(get_ind(processa_tendencia(pred1)),len(pred1))

print("----------------------")

# dados completos
print("Estatísticas completas")
show_stat(get_ind(processa_tendencia(tend_1)),len(tend_1))

Previsao com t = 0.1%
Nos dados, temos:
Queda: 0 - 0.0%
Estabilidade: 2819 - 100.0%
Aumento: 0 - 0.0%
----------------------
Estatísticas completas
Nos dados, temos:
Queda: 3248 - 34.6%
Estabilidade: 3982 - 42.4%
Aumento: 2166 - 23.1%


In [ ]:
# t = 0.05%
print("Previsao com t = 0.15%")
show_stat(get_ind(processa_tendencia(pred15)),len(pred15))

print("----------------------")

# dados completos
print("Estatísticas completas")
show_stat(get_ind(processa_tendencia(tend_15)),len(tend_15))

Previsao com t = 0.15%
Nos dados, temos:
Queda: 0 - 0.0%
Estabilidade: 2819 - 100.0%
Aumento: 0 - 0.0%
----------------------
Estatísticas completas
Nos dados, temos:
Queda: 2474 - 26.3%
Estabilidade: 5279 - 56.2%
Aumento: 1643 - 17.5%


In [ ]:
# 1) 2%

# Dados do consumo 
from load_data import get_consumo 
tend_2 = get_consumo(PATH, 0.02)

# dados completos
print("Estatísticas completas [t=2%]")
show_stat(get_ind(processa_tendencia(tend_2)),len(tend_2))

# separando conjuntos de treino e teste (70% e 30%)
X_train, X_test, y_train, y_test = train_test_split(df, tend_2,test_size=0.3)

#vamos definir o modelo
model = keras.Sequential()
model.add(Dense(12, input_shape=(12,), activation="softmax"))
model.add(Dense(3, activation="softmax"))

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

model.fit(X_train, y_train, epochs=10) 

# Avaliacao
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

predictions2 = model.predict(X_test)

pred2 = [list(i) for i in predictions2]
#pred = processa_tendencia(p)

Estatísticas completas [t=2%]
Nos dados, temos:
Queda: 1861 - 19.8%
Estabilidade: 6271 - 66.7%
Aumento: 1264 - 13.5%
Epoch 1/10
206/206 [==============================] - 1s 2ms/step - loss: 1.0728 - categorical_accuracy: 0.4893
Epoch 2/10
206/206 [==============================] - 0s 2ms/step - loss: 0.9265 - categorical_accuracy: 0.6678
Epoch 3/10
206/206 [==============================] - 0s 2ms/step - loss: 0.8762 - categorical_accuracy: 0.6678
Epoch 4/10
206/206 [==============================] - 0s 2ms/step - loss: 0.8624 - categorical_accuracy: 0.6678
Epoch 5/10
206/206 [==============================] - 0s 2ms/step - loss: 0.8589 - categorical_accuracy: 0.6678
Epoch 6/10
206/206 [==============================] - 0s 2ms/step - loss: 0.8579 - categorical_accuracy: 0.6678
Epoch 7/10
206/206 [==============================] - 0s 2ms/step - loss: 0.8576 - categorical_accuracy: 0.6678
Epoch 8/10
206/206 [==============================] - 0s 2ms/step - loss: 0.8574 - categorical_accu

In [ ]:
# t = 0.2%
print("Previsao com t = 0.2%")
show_stat(get_ind(processa_tendencia(pred2)),len(pred2))

print("----------------------")

# dados completos
print("Estatísticas completas")
show_stat(get_ind(processa_tendencia(tend_2)),len(tend_2))

Previsao com t = 0.2%
Nos dados, temos:
Queda: 0 - 0.0%
Estabilidade: 2819 - 100.0%
Aumento: 0 - 0.0%
----------------------
Estatísticas completas
Nos dados, temos:
Queda: 1861 - 19.8%
Estabilidade: 6271 - 66.7%
Aumento: 1264 - 13.5%


In [ ]:
# 1) 3%

# Dados do consumo 
from load_data import get_consumo 
tend_3 = get_consumo(PATH, 0.03)

# dados completos
print("Estatísticas completas [t=3%]")
show_stat(get_ind(processa_tendencia(tend_3)),len(tend_3))

# separando conjuntos de treino e teste (70% e 30%)
X_train, X_test, y_train, y_test = train_test_split(df, tend_3,test_size=0.3)

#vamos definir o modelo
model = keras.Sequential()
model.add(Dense(12, input_shape=(12,), activation="softmax"))
model.add(Dense(3, activation="softmax"))

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

model.fit(X_train, y_train, epochs=10) 

# Avaliacao
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

predictions3 = model.predict(X_test)

pred3 = [list(i) for i in predictions3]
#pred = processa_tendencia(p)

Estatísticas completas [t=3%]
Nos dados, temos:
Queda: 1216 - 12.9%
Estabilidade: 7284 - 77.5%
Aumento: 896 - 9.5%
Epoch 1/10
206/206 [==============================] - 1s 2ms/step - loss: 0.8878 - categorical_accuracy: 0.7237
Epoch 2/10
206/206 [==============================] - 0s 2ms/step - loss: 0.7586 - categorical_accuracy: 0.7646
Epoch 3/10
206/206 [==============================] - 0s 2ms/step - loss: 0.7091 - categorical_accuracy: 0.7760
Epoch 4/10
206/206 [==============================] - 0s 2ms/step - loss: 0.6906 - categorical_accuracy: 0.7760
Epoch 5/10
206/206 [==============================] - 1s 3ms/step - loss: 0.6829 - categorical_accuracy: 0.7760
Epoch 6/10
206/206 [==============================] - 1s 3ms/step - loss: 0.6795 - categorical_accuracy: 0.7760
Epoch 7/10
206/206 [==============================] - 1s 3ms/step - loss: 0.6779 - categorical_accuracy: 0.7760
Epoch 8/10
206/206 [==============================] - 0s 2ms/step - loss: 0.6771 - categorical_accura

In [ ]:
# 1) 4%

# Dados do consumo 
from load_data import get_consumo 
tend_4 = get_consumo(PATH, 0.04)

# dados completos
print("Estatísticas completas [t=4%]")
show_stat(get_ind(processa_tendencia(tend_4)),len(tend_4))

# separando conjuntos de treino e teste (70% e 30%)
X_train, X_test, y_train, y_test = train_test_split(df, tendencia,test_size=0.3)

#vamos definir o modelo
model = keras.Sequential()
model.add(Dense(12, input_shape=(12,), activation="softmax"))
model.add(Dense(3, activation="softmax"))

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

model.fit(X_train, y_train, epochs=10) 

# Avaliacao
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

predictions4 = model.predict(X_test)

pred4 = [list(i) for i in predictions4]
#pred = processa_tendencia(p)

Estatísticas completas [t=4%]
Nos dados, temos:
Queda: 937 - 10.0%
Estabilidade: 7708 - 82.0%
Aumento: 751 - 8.0%
Epoch 1/10
206/206 [==============================] - 1s 2ms/step - loss: 0.8883 - categorical_accuracy: 0.6877
Epoch 2/10
206/206 [==============================] - 0s 1ms/step - loss: 0.6982 - categorical_accuracy: 0.8119
Epoch 3/10
206/206 [==============================] - 0s 2ms/step - loss: 0.6125 - categorical_accuracy: 0.8416
Epoch 4/10
206/206 [==============================] - 0s 2ms/step - loss: 0.5743 - categorical_accuracy: 0.8416
Epoch 5/10
206/206 [==============================] - 0s 2ms/step - loss: 0.5568 - categorical_accuracy: 0.8416
Epoch 6/10
206/206 [==============================] - 0s 2ms/step - loss: 0.5479 - categorical_accuracy: 0.8416
Epoch 7/10
206/206 [==============================] - 0s 1ms/step - loss: 0.5432 - categorical_accuracy: 0.8416
Epoch 8/10
206/206 [==============================] - 1s 2ms/step - loss: 0.5405 - categorical_accurac

In [ ]:
# t = 0.3%
print("Previsao com t = 0.3%")
show_stat(get_ind(processa_tendencia(pred3)),len(pred3))

print("----------------------")

# dados completos
print("Estatísticas completas")
show_stat(get_ind(processa_tendencia(tend_3)),len(tend_3))

Previsao com t = 0.3%
Nos dados, temos:
Queda: 0 - 0.0%
Estabilidade: 2819 - 100.0%
Aumento: 0 - 0.0%
----------------------
Estatísticas completas
Nos dados, temos:
Queda: 1216 - 12.9%
Estabilidade: 7284 - 77.5%
Aumento: 896 - 9.5%


In [ ]:
# t = 1%
print("Previsao com t = 1%")
show_stat(get_ind(processa_tendencia(pred1)),len(pred1))

print("---------------------------")


# t = 2%
print("Previsao com t = 2%")
show_stat(get_ind(processa_tendencia(pred2)),len(pred2))

print("---------------------------")

# t = 3%
print("Previsao com t = 3%")
show_stat(get_ind(processa_tendencia(pred3)),len(pred3))

print("---------------------------")


Previsao com t = 1%
Nos dados, temos:
Queda: 0 - 0.0%
Estabilidade: 2819 - 100.0%
Aumento: 0 - 0.0%
---------------------------
Previsao com t = 2%
Nos dados, temos:
Queda: 0 - 0.0%
Estabilidade: 2819 - 100.0%
Aumento: 0 - 0.0%
---------------------------
Previsao com t = 3%
Nos dados, temos:
Queda: 0 - 0.0%
Estabilidade: 2819 - 100.0%
Aumento: 0 - 0.0%
---------------------------
